## Exploring Hacker News Posts

In this project, we'll work with a data set of submissions to popular technology site Hacker News. 

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

The data we used here has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

   + id: The unique identifier from Hacker News for the post
   + title: The title of the post
   + url: The URL that the posts links to, if it the post has a URL
   + num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
   + num_comments: The number of comments that were made on the post
   + author: The username of the person who submitted the post
   + created_at: The date and time at which the post was submitted

Let's read the data and take a look at the first few rows.

In [1]:
from csv import reader
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question.

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll compare these two types of posts to determine the following:

   + Do Ask HN or Show HN receive more comments on average?
   + Do posts created at a certain time receive more comments on average?


## Removing Headers
Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers. Let's remove that first row next.

In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [3]:
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Extracting Ask HN and Show HN Posts
Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [4]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    Newtitle = title.lower()
    if Newtitle.startswith('ask hn'):
        ask_posts.append(row)
    elif Newtitle.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

In [5]:
print(len(ask_posts),",", len(show_posts), ",", len(other_posts))

1744 , 1162 , 17194


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

We just separated the "ask posts" and the "show posts" into two list of lists named ask_posts and show_posts. Next, let's determine if ask posts or show posts receive more comments on average.

In [6]:
total_ask_comments = 0
for row in ask_posts:
    total_ask_comments = total_ask_comments + int(row[4])
avg_ask_comments = total_ask_comments/len(ask_posts)
print(avg_ask_comments)

14.038417431192661


In [7]:
total_show_comments = 0
for row in show_posts:
    total_show_comments = total_show_comments + int(row[4])
avg_show_comments = total_show_comments/len(show_posts)
print(avg_show_comments)

10.31669535283993


Show posts receice less comments than ask posts as we show here.

## Finding the Amount of Ask Posts and Comments by Hour Created

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

   + Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
   + Calculate the average number of comments ask posts receive by hour created.

In [8]:
import datetime as dt
result_list = []
for post in ask_posts:
    lis = []
    lis.append(post[6])
    lis.append(int(post[4]))
    result_list.append(lis)

In [9]:
#Generate a frequency table for comments created at each hour 
counts_by_hour = {}
comments_by_hour = {}
for lis in result_list:
    dt_object = dt.datetime.strptime(lis[0], "%m/%d/%Y %H:%M")
    hour = dt_object.strftime("%H")
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] = comments_by_hour[hour] + lis[1]
    else:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = lis[1]
    

In [10]:
print(counts_by_hour)

{'04': 47, '05': 46, '12': 73, '15': 116, '08': 48, '02': 58, '06': 44, '23': 68, '22': 71, '11': 58, '03': 54, '13': 85, '14': 107, '20': 80, '19': 110, '17': 100, '18': 109, '00': 55, '09': 45, '10': 59, '16': 108, '21': 109, '01': 60, '07': 34}


In [11]:
print(comments_by_hour)

{'04': 337, '05': 464, '12': 687, '15': 4477, '08': 492, '02': 1381, '06': 397, '23': 543, '22': 479, '11': 641, '03': 421, '13': 1253, '14': 1416, '20': 1722, '19': 1188, '17': 1146, '18': 1439, '00': 447, '09': 251, '10': 793, '16': 1814, '21': 1745, '01': 683, '07': 267}


## Calculating the Average Number of Comments for Ask HN Posts by Hour

Next, we'll use these two dictionaries we generated last step to calculate the average number of comments for posts created during each hour of the day.

In [12]:
#calculate the average number of comments 
#per post for posts created during each hour of the day
avg_by_hour = []
for count in counts_by_hour:
    avg = comments_by_hour[count]/counts_by_hour[count]
    avg_by_hour.append([count, avg])

In [13]:
print(avg_by_hour)

[['04', 7.170212765957447], ['05', 10.08695652173913], ['12', 9.41095890410959], ['15', 38.5948275862069], ['08', 10.25], ['02', 23.810344827586206], ['06', 9.022727272727273], ['23', 7.985294117647059], ['22', 6.746478873239437], ['11', 11.051724137931034], ['03', 7.796296296296297], ['13', 14.741176470588234], ['14', 13.233644859813085], ['20', 21.525], ['19', 10.8], ['17', 11.46], ['18', 13.20183486238532], ['00', 8.127272727272727], ['09', 5.5777777777777775], ['10', 13.440677966101696], ['16', 16.796296296296298], ['21', 16.009174311926607], ['01', 11.383333333333333], ['07', 7.852941176470588]]


## Sorting and Printing Values

Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [14]:
swap_avg_by_hour = []
for avg in avg_by_hour:
    swap_avg_by_hour.append([avg[1], avg[0]])
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

In [15]:
print('Top 5 Hours for Ask Posts Comments')
for i in range(5):
    dt_object_new=dt.datetime.strptime(sorted_swap[i][1],"%H")
    hour_new = dt_object_new.strftime("%H:%M")
    print("{}: {:.2f} average comments per post".format(hour_new,sorted_swap[i][0]))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


According to the result, for Ask HN we should create a post around 15:00 to have a higher chance of receiving comments.